In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import get_data
import preprocess
import base_models
from base_models import load_base_modeled
import learner
import evaluate_results

import matplotlib.pyplot as plt
import time
import tensorflow as tf

In [ ]:
train_ds = load_base_modeled(base_models.vgg16_label, get_data.train_label)
valid_ds = load_base_modeled(base_models.vgg16_label, get_data.valid_label)
test_ds = load_base_modeled(base_models.vgg16_label, get_data.test_label)
input_shape = train_ds.element_spec[0].shape[1:]
print(input_shape)

In [ ]:
def hyperparameters_to_string(model_name, n_epochs, batch_size, steps_per_epoch):
    params = {"model": model_name, "epochs": n_epochs, "batch-size": batch_size, "steps-per-epoch": steps_per_epoch}
    return "_".join([f"{k}={params[k]}" for k in params])

In [ ]:
def evaluation_figs(history, results, label):
    fig,axs = plt.subplots(1, 3, figsize=(20, 6))
    evaluate_results.training_accuracy_plot(history, model_label=label, ax=axs[0])
    evaluate_results.training_loss_plot(history, model_label=label, ax=axs[1])
    evaluate_results.confusion_matrix(results, model_label=label, ax=axs[2])
    fig.suptitle(label)

def evaluate_hyperparameters(model_fn, n_epochs, batch_size, steps_per_epoch, n_samples=1, print_samples=False, verbose=0):
    label = hyperparameters_to_string(model_fn.__name__.replace("_", "-"), n_epochs, batch_size, steps_per_epoch)
    history, results = [], []
    if batch_size is not None:
        this_test_ds = test_ds.unbatch().batch(batch_size)
        this_test_ds.file_paths = test_ds.file_paths
    else: this_test_ds = test_ds

    t0 = time.time()
    for i in range(n_samples):
        model = model_fn(input_shape)  # Recreate the model each time for independent samples
        if print_samples: print(f"{label} ROUND {i}/{n_samples}")
        this_history = learner.train(model, n_epochs, train_ds.repeat(), valid_ds,
            steps_per_epoch=steps_per_epoch, verbose=verbose).history
        history.append(this_history)
        this_results = learner.test(model, this_test_ds, verbose=verbose)
        results.append(this_results)
        # evaluation_figs(this_history, this_results, label+" "+str(i))
    t1 = time.time()
    history = evaluate_results.combine_history(history)
    results = evaluate_results.combine_results(results)
    print(f"{label} ({(t1-t0):.2f}s)")
    evaluate_results.print_test_results(results)
    evaluation_figs(history, results, label)
    evaluate_results.generate_misclass_files(results, model_label=label)

Experiment 1: effect of varying epochs vs. steps_per_epoch while keeping the product constant

In [ ]:
n_samples = 30
evaluate_hyperparameters(learner.original_addon, 50, 32, 14, n_samples=n_samples)
evaluate_hyperparameters(learner.original_addon, 100, 32, 7, n_samples=n_samples)
evaluate_hyperparameters(learner.original_addon, 25, 32, 28, n_samples=n_samples)

No, not really. Lowering the number of epochs makes the loss spikes much more evident, though this just seems to be how the averaging turns out.

Experiment 2: effect of varying batch size while keeping all else constant

In [ ]:
evaluate_hyperparameters(learner.original_addon, 50, 32, 14, n_samples=n_samples)
evaluate_hyperparameters(learner.original_addon, 50, 16, 14, n_samples=n_samples)
evaluate_hyperparameters(learner.original_addon, 50, 64, 14, n_samples=n_samples)

Again, not much of an effect. Maybe slightly better with smaller batch sizes. We can just stick with the defaults then.

How many epochs?

In [ ]:
spe = tf.data.experimental.cardinality(train_ds).numpy()/preprocess.augmentation_reps
evaluate_hyperparameters(learner.original_addon, 50, None, spe, n_samples=3)
evaluate_hyperparameters(learner.original_addon, 500, None, spe, n_samples=3)

In [ ]:
evaluate_hyperparameters(learner.original_addon, 100, None, spe, n_samples=3)

So it seems that 50 epochs is pretty good.